In [15]:
import os
import numpy as np
from scipy.io import loadmat
import h5py

In [16]:
import numpy as np
import h5py

print("NumPy:", np.__version__)
print("h5py:", h5py.__version__)

NumPy: 1.26.4
h5py: 3.11.0


In [18]:
mat_path = r"E:\Masters Thesis\Data\TVSD\MonkeyF\THINGS_normMUA.mat"

with h5py.File(mat_path, "r") as f:
    print("Top-level keys in file:")
    for k in f.keys():
        print("  ", k)

Top-level keys in file:
   SNR
   SNR_max
   lats
   oracle
   reliab
   tb
   test_MUA
   test_MUA_reps
   train_MUA


In [1]:
def inspect_h5(group, indent=0, max_depth=10):
    """Recursively print structure of an HDF5 file/group (datasets, shapes, dtypes, attrs)."""
    ind = " " * indent

    # Show attributes on this group/file
    if len(group.attrs) > 0:
        print(f"{ind}Attributes:")
        for k, v in group.attrs.items():
            print(f"{ind}  @{k} = {v}")

    for key in group.keys():
        item = group[key]
        if isinstance(item, h5py.Group):
            print(f"{ind}{key}/  (GROUP)")
            if indent // 2 < max_depth:
                inspect_h5(item, indent + 2, max_depth=max_depth)
            else:
                print(f"{ind}  (max depth reached here)")
        elif isinstance(item, h5py.Dataset):
            print(f"{ind}{key}  (DATASET) shape={item.shape}, dtype={item.dtype}")
            if len(item.attrs) > 0:
                for ak, av in item.attrs.items():
                    print(f"{ind}  @{ak} = {av}")
        else:
            print(f"{ind}{key}: unknown type {type(item)}")


In [20]:
with h5py.File(mat_path, "r") as f:
    print("=== FULL HDF5 STRUCTURE ===")
    inspect_h5(f, indent=0, max_depth=8)


=== FULL HDF5 STRUCTURE ===
SNR  (DATASET) shape=(4, 1024), dtype=float64
  @MATLAB_class = b'double'
SNR_max  (DATASET) shape=(1024, 1), dtype=float64
  @MATLAB_class = b'double'
lats  (DATASET) shape=(4, 1024), dtype=float64
  @MATLAB_class = b'double'
oracle  (DATASET) shape=(1024, 1), dtype=float64
  @MATLAB_class = b'double'
reliab  (DATASET) shape=(435, 1024), dtype=float64
  @MATLAB_class = b'double'
tb  (DATASET) shape=(300, 1), dtype=float64
  @MATLAB_class = b'double'
test_MUA  (DATASET) shape=(100, 1024), dtype=float64
  @MATLAB_class = b'double'
test_MUA_reps  (DATASET) shape=(30, 100, 1024), dtype=float64
  @MATLAB_class = b'double'
train_MUA  (DATASET) shape=(22248, 1024), dtype=float64
  @MATLAB_class = b'double'


### TVSD – `THINGS_normMUA.mat` (Monkey F): HDF5 structure

This file contains **pre-processed, normalized multi-unit activity (MUA)** and quality metrics,
already averaged over a stimulus-response time window. Each row (or index) refers to **stimuli**,
each column to **electrodes** (1024 total). :contentReference[oaicite:0]{index=0}  

---

#### Overview table

| Dataset name      | Shape             | Axes (Python)                              | Meaning (short)                                   |
|-------------------|-------------------|--------------------------------------------|---------------------------------------------------|
| `SNR`             | `(4, 1024)`       | day × electrode                            | Signal-to-noise ratio per day and electrode       |
| `SNR_max`         | `(1024, 1)`       | electrode × 1                              | Max SNR per electrode                             |
| `lats`            | `(4, 1024)`       | day × electrode                            | Response latency (ms) per day and electrode       |
| `oracle`          | `(1024, 1)`       | electrode × 1                              | Oracle noise ceiling per electrode                |
| `reliab`          | `(435, 1024)`     | pair-of-reps × electrode                   | Pairwise test-repetition reliability              |
| `tb`              | `(300, 1)`        | time-samples × 1                           | Time vector (ms) used to define averaging window  |
| `test_MUA`        | `(100, 1024)`     | test-image × electrode                     | Mean normalized response to each **test** image   |
| `test_MUA_reps`   | `(30, 100, 1024)` | repetition × test-image × electrode        | Single-trial test responses                       |
| `train_MUA`       | `(22248, 1024)`   | train-image × electrode                    | Mean normalized response to each **train** image  |

--